In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import os
import sqlite3
import numpy as np

# --- 1. 경로 설정 ---

# ED 데이터셋 폴더 경로 (vitalsign.csv가 있는 것을 확인한 경로)
ed_base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/mimic-iv-ed-2.2/'

# HOSP/ICU 데이터베이스 파일 경로
db_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스/MIMIC4-hosp-icu.db'

# --- 2. 데이터 로드 및 코호트 생성 ---
print("데이터 로드 중...")

try:
    # --- 2-1. ED 데이터 로드 (CSV) ---
    edstays_csv_path = os.path.join(ed_base_path, 'edstays.csv')
    df_edstays = pd.read_csv(edstays_csv_path,
                             dtype={'subject_id': int, 'hadm_id': 'Int64', 'stay_id': int})
    print(f"edstays.csv 로드 완료 (Shape: {df_edstays.shape})")

    # --- 2-2. HOSP 데이터 로드 (SQLite) ---
    conn = sqlite3.connect(db_path) # DB 연결
    query = "SELECT subject_id, hadm_id, icd_code, icd_version FROM diagnoses_icd"
    df_diagnoses_icd = pd.read_sql_query(query, conn)
    conn.close() # DB 연결 해제

    df_diagnoses_icd['icd_code'] = df_diagnoses_icd['icd_code'].astype(str)
    print(f"diagnoses_icd DB 테이블 로드 완료 (Shape: {df_diagnoses_icd.shape})")

    # --- 3. STEMI 진단 코드 정의 및 필터링 ---
    # (이하 로직은 데이터 로드가 모두 성공해야 실행됩니다)

    # ICD-10
    stemi_icd10 = ['I210', 'I211', 'I212', 'I213']
    # ICD-9
    stemi_icd9 = ['4100', '4101', '4102', '4103', '4104', '4105', '4106', '4108']

    df_diagnoses_icd['icd_code_cleaned'] = df_diagnoses_icd['icd_code'].str.replace('.', '', regex=False).str.upper()

    mask_stemi = (
        (df_diagnoses_icd['icd_version'] == 9) &
        (df_diagnoses_icd['icd_code_cleaned'].str.startswith(tuple(stemi_icd9))) &
        (df_diagnoses_icd['icd_code_cleaned'].str.endswith('1'))
    ) | (
        (df_diagnoses_icd['icd_version'] == 10) &
        (df_diagnoses_icd['icd_code_cleaned'].str.startswith(tuple(stemi_icd10)))
    )

    stemi_diagnoses = df_diagnoses_icd[mask_stemi]
    stemi_admissions = stemi_diagnoses[['subject_id', 'hadm_id']].drop_duplicates()

    print(f"\n총 STEMI 진단 입원(hadm_id) 건수: {len(stemi_admissions)}")

    # --- 4. ED 방문 기록과 STEMI 입원 기록 병합 (코호트 생성) ---
    print("ED 방문 기록과 STEMI 입원 기록 병합 중...")

    df_edstays_with_admission = df_edstays.dropna(subset=['hadm_id'])

    cohort_df = pd.merge(
        df_edstays_with_admission,
        stemi_admissions,
        on=['subject_id', 'hadm_id']
    )

    print("\n--- 🚀 최종 코호트 생성 완료 ---")
    print(f"분석 대상 코호트 (ED -> STEMI 입원) 환자(stay_id) 수: {len(cohort_df)}")
    print(f"코호트 Shape: {cohort_df.shape}")
    print("\n[코호트 데이터 샘플 (상위 5건)]")
    print(cohort_df.head())


# --- 5. 오류 처리 ---
except FileNotFoundError as e:
    print("\n" + "="*50)
    print(f"--- 🚨 파일 로드 오류 (CSV) ---")
    print(f"오류: '{e.filename}' 파일을 찾을 수 없습니다.")
    print(f"경로: '{ed_base_path}'")
    print("팁: Colab 왼쪽 파일(📁) 탭에서 해당 경로에 파일이 있는지, 파일 이름이 정확한지(대소문자) 확인하세요.")
    print("="*50 + "\n")

except pd.io.sql.DatabaseError as e:
    print(f"--- 🚨 DB 연결 오류 (SQLite) ---")
    print(f"오류: DB 연결이나 'diagnoses_icd' 테이블 쿼리에 실패했습니다.")
    print(f"경로: '{db_path}'")
    print(f"상세 오류: {e}")

except Exception as e:
    print(f"--- 🚨 알 수 없는 오류 ---")
    print(f"오류 발생: {e}")
    # 파일 로드에 성공했더라도, 이후 로직(3, 4단계)에서 오류가 발생할 수 있습니다.

데이터 로드 중...
edstays.csv 로드 완료 (Shape: (425087, 9))
diagnoses_icd DB 테이블 로드 완료 (Shape: (6364488, 4))

총 STEMI 진단 입원(hadm_id) 건수: 2533
ED 방문 기록과 STEMI 입원 기록 병합 중...

--- 🚀 최종 코호트 생성 완료 ---
분석 대상 코호트 (ED -> STEMI 입원) 환자(stay_id) 수: 836
코호트 Shape: (836, 9)

[코호트 데이터 샘플 (상위 5건)]
   subject_id   hadm_id   stay_id               intime              outtime  \
0    10010058  26359957  39919549  2147-11-18 00:50:00  2147-11-18 03:19:00   
1    10012438  22764825  35750515  2178-06-07 19:33:00  2178-06-07 21:57:00   
2    10013310  21243435  31736720  2153-05-26 08:56:00  2153-05-26 14:18:39   
3    10013310  27682188  35160955  2153-05-06 10:21:00  2153-05-06 18:28:00   
4    10033552  26487381  32542054  2136-02-29 22:09:00  2136-03-01 02:26:37   

  gender                      race arrival_transport disposition  
0      M  ASIAN - SOUTH EAST ASIAN         AMBULANCE    ADMITTED  
1      M                     WHITE         AMBULANCE    ADMITTED  
2      F             BLACK/AFRICAN           UNKN